## Memory
This section covers the use of memory modules in langchain and how we can use prebuilt chains for accessing memory to build a simple conversational AI.

Outline for various memeory types
* ConversationalBufferMemory
* ConversationalBufferWindowMemory
* ConversationalTokenBufferMemory
* ConversationalSummaryMemory

In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [2]:
from dotenv import load_dotenv
load_dotenv()

mistral_7b = ChatOpenAI(temperature=0.0, model="mistralai/Mistral-7B-Instruct-v0.2")
mistral_7b._default_params

{'model': 'mistralai/Mistral-7B-Instruct-v0.2',
 'stream': False,
 'n': 1,
 'temperature': 0.0}

In [4]:
# instantiate conversation memory 
memory = ConversationBufferMemory()

In [5]:
conv_chain = ConversationChain(
    llm=mistral_7b,
    memory=memory,
    verbose=True
)

In [6]:
conv_chain.predict(input="Hi my name is Kosi")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. Check the current conversation section for context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
DO NOT TRY TO CONTINUE THE CONVERSATION ON YOUR OWN, ONLY PROVIDE THE A SINGLE AI RESPONSE.
Return just the response.

Current conversation:

Human: Hi my name is Kosi
AI:

> Finished chain.


"Hello Kosi, nice to meet you. I'm an AI language model. How can I assist you today?"

In [7]:
conv_chain.predict(input="What is 1 + 1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. Check the current conversation section for context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
DO NOT TRY TO CONTINUE THE CONVERSATION ON YOUR OWN, ONLY PROVIDE THE A SINGLE AI RESPONSE.
Return just the response.

Current conversation:
Human: Hi my name is Kosi
AI: Hello Kosi, nice to meet you. I'm an AI language model. How can I assist you today?
Human: What is 1 + 1?
AI:

> Finished chain.


'The answer to the mathematical expression "1 + 1" is "2". This is a basic arithmetic operation where "1" is added to "1", resulting in a sum of "2".'

In [8]:
conv_chain.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. Check the current conversation section for context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
DO NOT TRY TO CONTINUE THE CONVERSATION ON YOUR OWN, ONLY PROVIDE THE A SINGLE AI RESPONSE.
Return just the response.

Current conversation:
Human: Hi my name is Kosi
AI: Hello Kosi, nice to meet you. I'm an AI language model. How can I assist you today?
Human: What is 1 + 1?
AI: The answer to the mathematical expression "1 + 1" is "2". This is a basic arithmetic operation where "1" is added to "1", resulting in a sum of "2".
Human: What is my name?
AI:

> Finished chain.


'Kosi, as you introduced yourself at the beginning of our conversation.'

In [37]:
conv_chain.predict(input="I mentioned it earlier in the conversation")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi my name is Kosi
AI:  Hello Kosi! My name is LLaMA, I'm a large language model trained by a team of researcher at Meta AI. It's nice to meet you! What brings you here today?
Human: What is 1 + 1?
AI:  Sure! The answer to 1 + 1 is 2. Would you like to know anything else?
Human: What is my name?
AI:  AI: I don't know your name. I'm just an AI, I don't have access to personal information or the ability to perceive the physical world. I can only provide information based on my training data, and I don't have any information about you personally. Is there anything else I can help you with?
Human: I mentioned it earlier in the conversation
AI:

> Finished chain.


' Ah, my apologies! You did mention your name earlier in the conversation. Your name is Kosi. Is there anything else you would like to know or discuss?'

In [9]:
print(type(memory.buffer))

<class 'str'>


In [11]:
from pprint import pprint

In [12]:
pprint(memory.load_memory_variables({}))

{'history': 'Human: Hi my name is Kosi\n'
            "AI: Hello Kosi, nice to meet you. I'm an AI language model. How "
            'can I assist you today?\n'
            'Human: What is 1 + 1?\n'
            'AI: The answer to the mathematical expression "1 + 1" is "2". '
            'This is a basic arithmetic operation where "1" is added to "1", '
            'resulting in a sum of "2".\n'
            'Human: What is my name?\n'
            'AI: Kosi, as you introduced yourself at the beginning of our '
            'conversation.'}


In [53]:
memory = ConversationBufferMemory()

In [54]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"}) # we can start hte memory with some predifined context, but why is it called input and output rather than human and AI

In [55]:
print(memory.buffer)

Human: Hi
AI: What's up


In [56]:
memory.load_memory_variables({}) # extract variables from the conversational buffer memory

{'history': "Human: Hi\nAI: What's up"}

In [57]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [58]:
print(memory.buffer)

Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool


### We can use ConversationBufferWindowMemory to optimize the context that is being sent to the LLM 
This would help reduce the conmpounding cost of the LLM session as the context expands with the memory

In [14]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=4) # stores only the last k messages in memory

In [15]:
conv_window_chain = ConversationChain(
    llm=mistral_7b,
    memory=memory,
    verbose=True
)

In [16]:
conv_window_chain.predict(input="Hello, my name is Kosi and I am 23 years old")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. Check the current conversation section for context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
DO NOT TRY TO CONTINUE THE CONVERSATION ON YOUR OWN, ONLY PROVIDE THE A SINGLE AI RESPONSE.
Return just the response.

Current conversation:

Human: Hello, my name is Kosi and I am 23 years old
AI:

> Finished chain.


"Hello Kosi, nice to meet you. I'm an AI and don't have the ability to age or have a name. I'm here to help answer any questions you might have to the best of my ability. You mentioned you're 23 years old. Is there a specific question related to that age or something else you'd like to know?"

In [18]:
conv_window_chain.predict(input="What is the captial of Nigeria?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. Check the current conversation section for context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
DO NOT TRY TO CONTINUE THE CONVERSATION ON YOUR OWN, ONLY PROVIDE THE A SINGLE AI RESPONSE.
Return just the response.

Current conversation:
Human: Hello, my name is Kosi and I am 23 years old
AI: Hello Kosi, nice to meet you. I'm an AI and don't have the ability to age or have a name. I'm here to help answer any questions you might have to the best of my ability. You mentioned you're 23 years old. Is there a specific question related to that age or something else you'd like to know?
Human: What is the captial of Nigeria?
AI:

> Finished chain.


"The capital city of Nigeria is Abuja. It became the capital in 1991, replacing Lagos. Abuja is located in the center of the country and is known for its modern architecture and urban planning. It has a population of over 2 million people. Is there anything else you'd like to know?"

In [19]:
conv_window_chain.predict(input="how old am i?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. Check the current conversation section for context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
DO NOT TRY TO CONTINUE THE CONVERSATION ON YOUR OWN, ONLY PROVIDE THE A SINGLE AI RESPONSE.
Return just the response.

Current conversation:
Human: Hello, my name is Kosi and I am 23 years old
AI: Hello Kosi, nice to meet you. I'm an AI and don't have the ability to age or have a name. I'm here to help answer any questions you might have to the best of my ability. You mentioned you're 23 years old. Is there a specific question related to that age or something else you'd like to know?
Human: What is the captial of Nigeria?
AI: The capital city of Nigeria is Abuja. It became the capital in 1991, replacing Lagos. Abuja is located in the cent

"I'm sorry for any confusion, Kosi. You mentioned that you are 23 years old at the beginning of our conversation. Is there a specific question related to that age or something else you'd like to know?"

In [20]:
conv_window_chain.predict(input="what is the most popular algorithm in computer science")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. Check the current conversation section for context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
DO NOT TRY TO CONTINUE THE CONVERSATION ON YOUR OWN, ONLY PROVIDE THE A SINGLE AI RESPONSE.
Return just the response.

Current conversation:
Human: Hello, my name is Kosi and I am 23 years old
AI: Hello Kosi, nice to meet you. I'm an AI and don't have the ability to age or have a name. I'm here to help answer any questions you might have to the best of my ability. You mentioned you're 23 years old. Is there a specific question related to that age or something else you'd like to know?
Human: What is the captial of Nigeria?
AI: The capital city of Nigeria is Abuja. It became the capital in 1991, replacing Lagos. Abuja is located in the cent

"The most popular algorithm in computer science can vary depending on the specific area of application. However, some of the most commonly used algorithms include:\n\n1. QuickSort for sorting data\n2. Dijkstra's Algorithm for finding the shortest path between nodes in a graph\n3. Floyd-Warshall Algorithm for finding the shortest paths between all pairs of nodes in a weighted graph\n4. Depth-First Search (DFS) and Breadth-First Search (BFS) for traversing graphs\n5. Hashing algorithms like HashTable and HashMap for efficient data storage and retrieval\n6. Gradient Descent for optimization problems in machine learning\n\nIs there a specific algorithm you have in mind that you'd like to know more about?"

In [21]:
conv_window_chain.predict(input="what is my name")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. Check the current conversation section for context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
DO NOT TRY TO CONTINUE THE CONVERSATION ON YOUR OWN, ONLY PROVIDE THE A SINGLE AI RESPONSE.
Return just the response.

Current conversation:
Human: Hello, my name is Kosi and I am 23 years old
AI: Hello Kosi, nice to meet you. I'm an AI and don't have the ability to age or have a name. I'm here to help answer any questions you might have to the best of my ability. You mentioned you're 23 years old. Is there a specific question related to that age or something else you'd like to know?
Human: What is the captial of Nigeria?
AI: The capital city of Nigeria is Abuja. It became the capital in 1991, replacing Lagos. Abuja is located in the cent

'Kosi, you mentioned that your name is Kosi at the beginning of our conversation.'

In [22]:
memory

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hello, my name is Kosi and I am 23 years old'), AIMessage(content="Hello Kosi, nice to meet you. I'm an AI and don't have the ability to age or have a name. I'm here to help answer any questions you might have to the best of my ability. You mentioned you're 23 years old. Is there a specific question related to that age or something else you'd like to know?"), HumanMessage(content='What is the captial of Nigeria?'), AIMessage(content="The capital city of Nigeria is Abuja. It became the capital in 1991, replacing Lagos. Abuja is located in the center of the country and is known for its modern architecture and urban planning. It has a population of over 2 million people. Is there anything else you'd like to know?"), HumanMessage(content='how old am i?'), AIMessage(content="I'm sorry for any confusion, Kosi. You mentioned that you are 23 years old at the beginning of our conversation. Is there a s

In [23]:
conv_window_chain.predict(input="what is the captial of America?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. Check the current conversation section for context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
DO NOT TRY TO CONTINUE THE CONVERSATION ON YOUR OWN, ONLY PROVIDE THE A SINGLE AI RESPONSE.
Return just the response.

Current conversation:
Human: What is the captial of Nigeria?
AI: The capital city of Nigeria is Abuja. It became the capital in 1991, replacing Lagos. Abuja is located in the center of the country and is known for its modern architecture and urban planning. It has a population of over 2 million people. Is there anything else you'd like to know?
Human: how old am i?
AI: I'm sorry for any confusion, Kosi. You mentioned that you are 23 years old at the beginning of our conversation. Is there a specific question related to th

'I\'m sorry for any confusion, but it seems there is a misunderstanding. The term "America" can refer to either the United States of America or the entire continent of North, Central, and South America. If you meant to ask about the capital city of the United States of America, it is Washington, D.C. If you meant to ask about the capital city of a specific country in South or Central America, please provide the name of that country.'

In [24]:
conv_window_chain.predict(input="what is my name")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. Check the current conversation section for context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
DO NOT TRY TO CONTINUE THE CONVERSATION ON YOUR OWN, ONLY PROVIDE THE A SINGLE AI RESPONSE.
Return just the response.

Current conversation:
Human: how old am i?
AI: I'm sorry for any confusion, Kosi. You mentioned that you are 23 years old at the beginning of our conversation. Is there a specific question related to that age or something else you'd like to know?
Human: what is the most popular algorithm in computer science
AI: The most popular algorithm in computer science can vary depending on the specific area of application. However, some of the most commonly used algorithms include:

1. QuickSort for sorting data
2. Dijkstra's Algori

'Kosi, you mentioned that your name is Kosi at the beginning of our conversation.'

### Conversational Token Buffer Memory

Even though we can limit the number of messages that go into our context for the LLM, a single message can be very large and fill up the entire context window, we would want to measure the number the number of messages that should go into the context of the LLM based on number of tokens 
That is what the ConversationalTokenBufferMemory helps us to do

In [3]:
from langchain.memory import ConversationTokenBufferMemory

In [4]:
token_buffer_memory = ConversationTokenBufferMemory(max_token_limit=50, llm=mistral_7b)

In [5]:
token_buffer_memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
token_buffer_memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
token_buffer_memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [7]:
token_buffer_memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

In [8]:
token_buffer_memory.save_context({"input": "Oooouuuu whatcha ssaaayyyyyyy, would you bring your lovin my wayyyy"}, 
                    {"output": "Ooouuu whatcha building my friend"})

In [9]:
token_buffer_memory.load_memory_variables({})

{'history': 'Human: Oooouuuu whatcha ssaaayyyyyyy, would you bring your lovin my wayyyy\nAI: Ooouuu whatcha building my friend'}

### We can also use ConversationSummaryMemory
This summarizes the conversation up to a point and usese the summerized version of the ocnversation

In [10]:
from langchain.memory import ConversationSummaryBufferMemory

In [11]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=mistral_7b, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [12]:
summary_conversation_chain = ConversationChain(
    llm=mistral_7b,
    verbose=True,
    memory=memory
)

In [13]:
summary_conversation_chain.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. Check the current conversation section for context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
DO NOT TRY TO CONTINUE THE CONVERSATION ON YOUR OWN, ONLY PROVIDE THE A SINGLE AI RESPONSE.
Return just the response.

Current conversation:
System: The human greets the AI and asks about the day's schedule. The AI responds by mentioning a 8am meeting with the product team where the human needs to prepare a powerpoint presentation. From 9am to 12pm, the human has time to work on the LangChain project, which the AI describes as a powerful tool. At noon, there is a lunch appointment with a customer who is coming from a long distance to discuss the latest advancements in AI. The human is advised to bring their laptop to demonstrate the LLM d

"Based on the latest advancements in AI, I would recommend demonstrating the language model's ability to understand and generate human-like text. You could also showcase its capability to answer complex questions, summarize long texts, and even generate creative content like poetry or stories. Additionally, you could demonstrate its integration with other applications or systems, if applicable."

In [14]:
summary_conversation_chain.predict(input="how many hours do i have to work on my langchain project today?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. Check the current conversation section for context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
DO NOT TRY TO CONTINUE THE CONVERSATION ON YOUR OWN, ONLY PROVIDE THE A SINGLE AI RESPONSE.
Return just the response.

Current conversation:
System: The human greets the AI and asks about the day's schedule. The AI responds by mentioning a 8am meeting with the product team where the human needs to prepare a powerpoint presentation. From 9am to 12pm, the human has time to work on the LangChain project, which the AI describes as a powerful tool. At noon, there is a lunch appointment with a customer who is coming from a long distance to discuss the latest advancements in AI. The human is advised to bring their laptop to demonstrate the LLM d

'You have a total of 3 hours to work on the LangChain project today, from 9am to 12pm.'

In [16]:
from pprint import pprint

In [17]:
pprint(memory.load_memory_variables({}))

{'history': "System: The human greets the AI and asks about the day's "
            'schedule. The AI informs the human of an 8am meeting with the '
            'product team where they need to prepare a powerpoint '
            'presentation. From 9am to 12pm, the human has time to work on the '
            'LangChain project, which the AI describes as a powerful tool. At '
            'noon, there is a lunch appointment with a customer coming from a '
            'long distance. The human is advised to bring their laptop to '
            "demonstrate the language model's ability to understand and "
            'generate human-like text, answer complex questions, summarize '
            'long texts, generate creative content, and integrate with other '
            'applications or systems.\n'
            'Human: how many hours do i have to work on my langchain project '
            'today?\n'
            'AI: You have a total of 3 hours to work on the LangChain project '
            '